#### NN Experiments

- Vanilla and Features NNs

In [ ]:
import numpy as np
from matplotlib import pyplot as plt 
import time as time
import sys
import platform
import psutil

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, Subset

from captum.attr import Saliency

from sklearn.model_selection import train_test_split

sys.path.append('..')

from prox_op import prox_op
from data_fcns import generate_raw_data
from data_fcns import vanilla_scaling
from data_fcns import compute_features

In [ ]:
# python version

print(sys.version)

In [ ]:
# get CPU info

print(platform.processor())
print(platform.machine())
print(platform.version())
print(platform.platform())
print(platform.uname())
print(platform.system())
print(str(round(psutil.virtual_memory().total / (1024.0 **3)))+" GB")


In [ ]:
# get GPU info

print(torch.cuda.is_available())

if torch.cuda.is_available():
    print(torch.cuda.device_count())
    print(torch.cuda.current_device())
    print(torch.cuda.device(0))
    print(torch.cuda.get_device_name(0))

#### Data Prep

##### Create data

In [ ]:
nn_type = "feature"  # vanilla or feature
data_dist = "unif"   # norm, unif, or both
unif_min = 0
unif_max = 20
min_len = 1000
max_len = 2000
num_vec = 10000
seed = 1

X, lengths, alphas, taus = generate_raw_data(data_dist, min_len, max_len, num_vec, unif_min, unif_max, seed)

In [ ]:
print(lengths.shape)
print(taus.shape)
print(alphas.shape)
print(X.shape)

##### preprocess data

In [ ]:
if nn_type == "vanilla":

    num_moments = 0 # needed for the NN selection cell
    M, yhat, zero_idx = vanilla_scaling(X, lengths, alphas, taus)
    
else:  # features NN
    num_moments = 10
    M, yhat, mus, zero_idx = compute_features(X, lengths, alphas, taus, num_moments)


In [ ]:
# remove any observations from dataset that have tau = 0 

if sum(zero_idx) > 0:
    M = M[~zero_idx,:]
    yhat = yhat[~zero_idx]
    mus = mus[~zero_idx]
    alphas = alphas[~zero_idx]
    taus = taus[~zero_idx]
    
if data_dist == "norm":
    num_norm_vec = M.shape[0]
elif data_dist == "unif":  
    num_norm_vec = 0
else: # both
    num_norm_vec = int(num_vec/2) - sum(np.where(zero_idx)[0] < int(num_vec/2))
    
print(num_norm_vec)

### NN

In [ ]:
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
class TauDataset(Dataset):
    def __init__(self, X, y, num_norm_vec):
        
        d = np.zeros(len(y))
        d[:num_norm_vec] = 1  # Gaussian - 1, Uniform - 0
          
        self.features = torch.Tensor(X)
        self.labels = torch.Tensor(y) 
        self.dist = torch.Tensor(d)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx,:], self.labels[idx], self.dist[idx], idx

In [ ]:
# create dataset
dataset = TauDataset(M,yhat,num_norm_vec)

print(dataset[0])
print(len(dataset))

In [ ]:
if data_dist == "both":

    # train-test split: 80-20, stratified sampling on distribution type
    # Gaussian: 1st half of indices, Uniform: 2nd half of indices 

    train_idx, test_idx = train_test_split(range(len(dataset)), test_size=0.20, random_state=0, 
                                       shuffle=True, stratify=dataset.dist)
    
else: # unif or norm data
    
    # train-test split: 80-20
    train_idx, test_idx = train_test_split(range(len(dataset)), test_size=0.20, random_state=0, shuffle=True)


In [ ]:
sum(dataset.dist[train_idx])

In [ ]:
sum(dataset.dist[test_idx])

In [ ]:
train_data = Subset(dataset, train_idx)
test_data = Subset(dataset, test_idx)

In [ ]:
# set up dataset iterator

train_batch_size = 32
test_batch_size = len(test_data)

train_loader = DataLoader(dataset=train_data, batch_size=train_batch_size, shuffle=True) 
test_loader = DataLoader(dataset=test_data, batch_size=test_batch_size, shuffle=True)

In [ ]:
print(len(train_loader))
print(len(test_loader))

#### Build NN

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
# first layer number of inputs
print(dataset[0][0].size())
layer1_size = M[0].shape[0]
layer1_size

In [ ]:
# set NN based on layer1_size

if layer1_size == (num_moments+3):  # features NN
    
    class NeuralNetwork(nn.Module):
        def __init__(self):
            super().__init__()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(num_moments+3, 25),  
                nn.ReLU(),    
                nn.Linear(25, 10),
                nn.ReLU(),
                nn.Linear(10, 1)
            )

        def forward(self, x):
            tau = self.linear_relu_stack(x) 
            return tau
        
elif (layer1_size == 2000) or (layer1_size == 100000):   # vanilla NN
    
    class NeuralNetwork(nn.Module):
        def __init__(self):
            super().__init__()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(layer1_size, 200),   
                nn.ReLU(),  
                nn.Linear(200, 100),
                nn.ReLU(),
                nn.Linear(100, 50), 
                nn.ReLU(),
                nn.Linear(50, 1) 
            )

        def forward(self, x):
            tau = self.linear_relu_stack(x)
            return tau
    
else:
    pass

model = NeuralNetwork().to(device)
print(model)

### Optimize NN

In [ ]:
learning_rate = 0.001
epochs = 5000

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)   

In [ ]:
# Train on CPU or GPU - use appropriate timing commands!

def train(dataloader, model, loss_fn, optimizer):
    #size = len(dataloader.dataset)
    
    # GPU timers
    #start_time = torch.cuda.Event(enable_timing=True)
    #end_time = torch.cuda.Event(enable_timing=True)
    
    model.train()
    
    num_batches = len(dataloader)
    avg_loss_per_obs = 0
    epoch_time = 0
    
    for batch, (X, y, dist, idx) in enumerate(dataloader):
        
        # CPU time 
        t1 = time.perf_counter()
        # GPU time
        #start_time.record()
           
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        
        # Compute prediction error
        pred = model(X)  
        loss = loss_fn(pred, y.unsqueeze(1))  

        # Backpropagation
        
        loss.backward()
        optimizer.step()

        # CPU time
        t2 = time.perf_counter()
        epoch_time += t2-t1
        # GPU time
        #end_time.record()
        #torch.cuda.synchronize()
        #epoch_time += (start_time.elapsed_time(end_time))/1000  # time unit is milliseconds
        
        avg_loss_per_obs += loss.item()
            
    avg_loss_per_obs /= num_batches
    print(f"Epoch train time: {epoch_time} seconds\n")
    print(f"Train Error: \n Avg loss (per obs): {avg_loss_per_obs:.2e} \n")   
    
    return avg_loss_per_obs, epoch_time

In [ ]:
# Testing on CPU or GPU - use appropriate timing commands!

def test(dataloader, model, loss_fn):
    
    #size = len(dataloader.dataset)
    
    # GPU timers
    #start_time = torch.cuda.Event(enable_timing=True)
    #end_time = torch.cuda.Event(enable_timing=True)
    
    num_batches = len(dataloader)
   
    model.eval()
    test_loss_all = 0
    test_loss_norm = 0
    test_loss_unif = 0
    test_loss_og = 0
    epoch_time = 0
    with torch.no_grad():
        for X, y, dist, idx in dataloader:
            X, y, dist, idx = X.to(device), y.to(device), dist.to(device), idx.to(device)
            
            # CPU time
            t1 = time.perf_counter()
            # GPU time
            #start_time.record()
            
            pred = model(X)
            
            # CPU time
            t2 = time.perf_counter()
            epoch_time += t2-t1
            # GPU time
            #end_time.record()
            #torch.cuda.synchronize()
            #epoch_time += (start_time.elapsed_time(end_time))/1000  # time unit is milliseconds
            
            test_loss_all += loss_fn(pred, y.unsqueeze(1)).item()
            test_loss_norm += loss_fn(pred[dist == 1], y[dist == 1].unsqueeze(1)).item() 
            test_loss_unif += loss_fn(pred[dist == 0], y[dist == 0].unsqueeze(1)).item()
    
            # additional code - loss on original tau.  If use GPU - must send pred to CPU

            if device == "cuda":
                pred_og_tau = pred.squeeze().cpu().numpy()
                idx = idx.cpu()
            else:
                pred_og_tau = pred.squeeze().numpy()
        
            if nn_type == "feature":

                pred_og_tau = np.add(pred_og_tau, mus[idx])
                pred_og_tau = np.multiply(alphas[idx], pred_og_tau)
                test_loss_og += loss_fn(torch.Tensor(pred_og_tau), torch.Tensor(taus[idx])).item()     
                    
            else:  # vanilla NN
                
                pred_og_tau = np.multiply(alphas[idx], pred_og_tau)
                test_loss_og += loss_fn(torch.Tensor(pred_og_tau), torch.Tensor(taus[idx])).item()             
                
    test_loss_all /= num_batches
    test_loss_norm /= num_batches
    test_loss_unif /= num_batches
    test_loss_og /= num_batches
    
    print(f"Epoch test time: {epoch_time} seconds\n")
    print(f"Test Error: \n Avg loss (per obs): {test_loss_all:.2e} \n") 
    print(f"Gaussian Test Error: \n Avg loss (per obs): {test_loss_norm:.2e} \n")
    print(f"Uniform Test Error: \n Avg loss (per obs): {test_loss_unif:.2e} \n")
    print(f"Original Tau Test Error: \n Avg loss (per obs): {test_loss_og:.2e} \n")   

    return test_loss_all, test_loss_norm, test_loss_unif, test_loss_og, epoch_time 

In [ ]:
len(test_loader.dataset)

In [ ]:
train_time = np.zeros(epochs)
test_time = np.zeros(epochs)

avg_loss_per_obs_test = np.zeros(epochs)
avg_loss_per_obs_train = np.zeros(epochs)

avg_loss_per_obs_test_norm = np.zeros(epochs)
avg_loss_per_obs_test_unif = np.zeros(epochs)
avg_loss_per_obs_test_og = np.zeros(epochs)

min_og_loss = 100
min_epoch = 0

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    avg_loss_per_obs_train[t], train_time[t] = train(train_loader, model, loss_fn, optimizer)
    avg_loss_per_obs_test[t], avg_loss_per_obs_test_norm[t], avg_loss_per_obs_test_unif[t], avg_loss_per_obs_test_og[t], test_time[t] = test(test_loader, model, loss_fn)

    # save model - at epoch with minimum original tau test error
    
    if avg_loss_per_obs_test_og[t] < min_og_loss:
        msd = model.state_dict()
        min_og_loss = avg_loss_per_obs_test_og[t]
        min_epoch = t+1
    
print("Done!")

In [ ]:
min_epoch

In [ ]:
# save model - at epoch with minimum original tau test error

output_path = "models/density/unif_0_20/len_1000_2000/"
torch.save(msd, output_path + "epoch_" + str(min_epoch) + "_nn.pt")

In [ ]:
# save losses and times for all epochs 

np.save(output_path + 'train_avgloss.npy', avg_loss_per_obs_train)
np.save(output_path + 'test_avgloss.npy', avg_loss_per_obs_test)
#np.save(output_path + 'test_norm_avgloss.npy', avg_loss_per_obs_test_norm)
#np.save(output_path + 'test_unif_avgloss.npy', avg_loss_per_obs_test_unif)
np.save(output_path + 'test_og_avgloss.npy', avg_loss_per_obs_test_og)

np.save(output_path + 'train_time.npy', train_time)
np.save(output_path + 'test_time.npy', test_time)


In [ ]:
# time computations

print(f"Total train time: {sum(train_time)} seconds")
print(f"Average train epoch time: {np.mean(train_time)} seconds\n")

print(f"Total test time: {sum(test_time)} seconds")
print(f"Average test epoch time: {np.mean(test_time)} seconds")